In [1]:
%pip install inference

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import gdown
import zipfile
import warnings
warnings.filterwarnings("ignore")
from inference_sdk import InferenceHTTPClient

In [3]:
download_dir = "data.zip"
drive_link = "https://drive.google.com/uc?id=1AE83WabDLpq1uwwYRxPLnQN2uDy7Ci22"

gdown.download(drive_link, output = download_dir, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1AE83WabDLpq1uwwYRxPLnQN2uDy7Ci22
From (redirected): https://drive.google.com/uc?id=1AE83WabDLpq1uwwYRxPLnQN2uDy7Ci22&confirm=t&uuid=1dc36241-d606-4b68-aca3-47da1274e082
To: /teamspace/studios/this_studio/CV_Project/data.zip
100%|██████████| 26.6M/26.6M [00:00<00:00, 31.1MB/s]


'data.zip'

In [4]:
data_dir = "data"

os.makedirs(data_dir, exist_ok=True)
os.makedirs("images", exist_ok=True)

with zipfile.ZipFile(download_dir, 'r') as zipped:
    zipped.extractall(data_dir)

In [5]:
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="5XagyPRtCr1rJQzvdDwl"
)

In [6]:
import os
import json
from PIL import Image, UnidentifiedImageError
import base64
from io import BytesIO

for i in range(1, 100):
    image_path = f"data/New_Metro_Data/{i}.jpeg"
    output_path = f"images/bounded_{i}.json"

    if not os.path.exists(image_path):
        print(f"❌ Skipping {image_path}: File does not exist.")
        continue

    try:
        with Image.open(image_path) as img:
            img = img.convert("RGB")  # Ensure valid format
            buffered = BytesIO()
            img.save(buffered, format="JPEG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

        result = client.run_workflow(
            workspace_name="rishav-qz1u7",
            workflow_id="detect-and-classify",
            images={"image": img_base64},
            use_cache=False
        )

        # Save result to images/{i}.json
        with open(output_path, "w") as f:
            json.dump(result, f, indent=2)

        print(f"✅ Saved result for {i}.jpeg to {output_path}")

    except UnidentifiedImageError:
        print(f"⚠️ Skipping {image_path}: Not a valid image file.")
    except Exception as e:
        print(f"🔥 Error processing {image_path}: {e}")


⚠️ Skipping data/New_Metro_Data/1.jpeg: Not a valid image file.
⚠️ Skipping data/New_Metro_Data/2.jpeg: Not a valid image file.
⚠️ Skipping data/New_Metro_Data/3.jpeg: Not a valid image file.
✅ Saved result for 4.jpeg to images/bounded_4.json
✅ Saved result for 5.jpeg to images/bounded_5.json
✅ Saved result for 6.jpeg to images/bounded_6.json
⚠️ Skipping data/New_Metro_Data/7.jpeg: Not a valid image file.
✅ Saved result for 8.jpeg to images/bounded_8.json
✅ Saved result for 9.jpeg to images/bounded_9.json
✅ Saved result for 10.jpeg to images/bounded_10.json
⚠️ Skipping data/New_Metro_Data/11.jpeg: Not a valid image file.
⚠️ Skipping data/New_Metro_Data/12.jpeg: Not a valid image file.
✅ Saved result for 13.jpeg to images/bounded_13.json
✅ Saved result for 14.jpeg to images/bounded_14.json
✅ Saved result for 15.jpeg to images/bounded_15.json
✅ Saved result for 16.jpeg to images/bounded_16.json
✅ Saved result for 17.jpeg to images/bounded_17.json
✅ Saved result for 18.jpeg to images/boun

In [7]:
import shutil

shutil.make_archive("bound_images", 'zip', "images")

'/teamspace/studios/this_studio/CV_Project/bound_images.zip'